In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ffc73d6ee1c55dced3cb8b6ebd0313223ce039d0754262da3438d8c8f891513a
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.csv('/content/bdiag.csv', inferSchema=True,header=True, sep=',')
df.show()

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|
+--------+---------+-----------+------------+---

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#Convertimos las variables categoricas en numeros ficticios
indexer = StringIndexer(inputCol= "diagnosis", outputCol = "diagnostico")
DATOS = indexer.fiwt(df).transform(df)
#eliminamos la variable original no codificada
DATOS2 = DATOS.drop(*('diagnosis','id'))
DATOS2.show()

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|diagnostico|
+-----------+------------+--------------+---------+-----------

In [ ]:
DATOS2.printSchema()

root
 |-- radius_mean: double (nullable = true)
 |-- texture_mean: double (nullable = true)
 |-- perimeter_mean: double (nullable = true)
 |-- area_mean: double (nullable = true)
 |-- smoothness_mean: double (nullable = true)
 |-- compactness_mean: double (nullable = true)
 |-- concavity_mean: double (nullable = true)
 |-- concave points_mean: double (nullable = true)
 |-- symmetry_mean: double (nullable = true)
 |-- fractal_dimension_mean: double (nullable = true)
 |-- radius_se: double (nullable = true)
 |-- texture_se: double (nullable = true)
 |-- perimeter_se: double (nullable = true)
 |-- area_se: double (nullable = true)
 |-- smoothness_se: double (nullable = true)
 |-- compactness_se: double (nullable = true)
 |-- concavity_se: double (nullable = true)
 |-- concave points_se: double (nullable = true)
 |-- symmetry_se: double (nullable = true)
 |-- fractal_dimension_se: double (nullable = true)
 |-- radius_worst: double (nullable = true)
 |-- texture_worst: double (nullable = tr

In [ ]:
#Observamos cuantos casos tenemos de cada clase
DATOS2.groupBy('diagnostico').count().show()

+-----------+-----+
|diagnostico|count|
+-----------+-----+
|        0.0|  357|
|        1.0|  212|
+-----------+-----+



In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

DATOS2.columns


['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst',
 'diagnostico']

In [ ]:
var_independientes = VectorAssembler(inputCols=['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst'],
 outputCol='features')

var_independientes

VectorAssembler_786a0c1b853c

In [ ]:
output = var_independientes.transform(DATOS2)
output.show()

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+--------------------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|diagnostico|            features|
+-----------+-------

In [ ]:
output.select("features","diagnostico").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|features                                                                                                                                                                                                                 |diagnostico|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|[17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189]      |1.0        |
|[20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667

In [ ]:
datos_regresion = output.select("features","diagnostico")

In [ ]:
train,test = datos_regresion.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(labelCol="diagnostico").fit(train)
log_reg


LogisticRegressionModel: uid=LogisticRegression_2fcae8a7f2d4, numClasses=2, numFeatures=30

In [ ]:
resultados_entrenamiento = log_reg.evaluate(train).predictions
resultados_entrenamiento.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------------------------------------+-----------+----------+
|features                                                                                                                                                                                                                   |diagnostico|rawPrediction                           |probability|prediction|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+----------------------------------------+-----------+----------+
|[6.981,13.43,43.79,143.5,0.117,0.07568,0.0,0.0,0.193,0.07818,0.2241,1.508,1.553,9.833,0.01019,0.01084,0.0

In [ ]:
resultados_entrenamiento.select(['diagnostico','prediction']).show(50)

+-----------+----------+
|diagnostico|prediction|
+-----------+----------+
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|


In [ ]:
#probamos los resultados con test
resultado_test = log_reg.evaluate(test).predictions
resultado_test.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+-----------+--------------------+-----------+----------+
|            features|diagnostico|       rawPrediction|probability|prediction|
+--------------------+-----------+--------------------+-----------+----------+
|[8.219,20.7,53.27...|        0.0|[1325.26318161805...|  [1.0,0.0]|       0.0|
|[8.597,18.6,54.09...|        0.0|[4138.27342968689...|  [1.0,0.0]|       0.0|
|[8.878,15.49,56.7...|        0.0|[1296.40144208279...|  [1.0,0.0]|       0.0|
|[9.295,13.9,59.96...|        0.0|[1359.44748722858...|  [1.0,0.0]|       0.0|
|[9.465,21.01,60.1...|        0.0|[1106.96693068332...|  [1.0,0.0]|       0.0|
|[9.504,12.44,60.3...|        0.0|[1213.77367298462...|  [1.0,0.0]|       0.0|
|[9.606,16.84,61.6...|        0.0|[1135.15091450694...|  [1.0,0.0]|       0.0|
|[9.668,18.1,61.06...|        0.0|[1425.78271974397...|  [1.0,0.0]|       0.0|
|[9.683,19.34,61.0...|        0.0|[1157.66632812016...|  [1.0,0.0]|       0.0|
|[9.72,18.22,60.73...|        0.0|[4498.89014623092.

In [ ]:
resultado_test.select(['diagnostico','prediction']).show()

+-----------+----------+
|diagnostico|prediction|
+-----------+----------+
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
|        0.0|       0.0|
+-----------+----------+
only showing top 20 rows



In [ ]:
tp = resultado_test[(resultado_test.diagnostico==1) & (resultado_test.prediction==1)].count()
tp

49

In [ ]:
tn = resultado_test[(resultado_test.diagnostico==0) & (resultado_test.prediction==0)].count()
tn

100

In [ ]:
fp = resultado_test[(resultado_test.diagnostico==0) & (resultado_test.prediction==1)].count()
fp

7

In [ ]:
fn = resultado_test[(resultado_test.diagnostico==1) & (resultado_test.prediction==0)].count()
fn

3

In [ ]:
accuracy = float((tp + tn)/ (resultado_test.count()))

accuracy

0.9371069182389937

In [ ]:
#Calculamos el area bajo la curva

resultado_test = log_reg.transform(test)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='diagnostico')
resultado_test.select('diagnostico','prediction')
AUC = my_eval.evaluate(resultado_test)
print("AUC score is : ",AUC)

AUC score is :  0.9384435657800143


In [ ]:
print("Coefficients: " + str(log_reg.coefficients))
print("Intercept: " + str(log_reg.intercept))


Coefficients: [-37.970110976669524,51.9537377417296,-5.073872885635785,-0.19615622689776688,1439.9839553776267,-9222.21702830727,3874.4978924084944,8869.116829008213,-2118.9154650758283,29096.803438794905,1474.338553828122,-472.34746071438417,84.74349609682288,6.303575269187163,14222.786736283022,-10328.808662117674,-7053.869687692159,71044.05526859201,-37563.891155918434,-111425.5537719771,76.32248496633102,37.49931804214339,6.771162602193385,0.4705384394823276,1618.1245150750285,-2567.90546113551,1580.5228661931185,1578.6266617816668,3848.5349805954224,20063.467577397183]
Intercept: -7203.076722964926


In [ ]:
#Si quisieramos crear una prediccion partiendo de una sola observacion, creamos el dataframe y los convertimos en vector
#Debemos incluir la variable diagnostico de igual manera.
new = spark.createDataFrame(
    [
        (11.08,18.83,73.3,361.6,0.1216,0.2154,0.1689,0.06367,0.2196,0.0795,0.2114,1.027,1.719,13.99,0.007405,0.04549,0.04588,0.01339,0.01738,0.004435,13.24,32.82,91.76,508.1,0.2184,0.9379,0.8402,0.2524,0.4154,0.1403,0.0),

    ],
    ['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst','diagnostico']
)
new.show()

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|diagnostico|
+-----------+------------+--------------+---------+-----------

In [ ]:
from pyspark.ml.feature import VectorAssembler

vector_nuevo = VectorAssembler(
    inputCols=['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst'],
    outputCol="features")
vector_new = vector_nuevo.transform(new)

vector_new.show()

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+--------------------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|diagnostico|            features|
+-----------+-------

In [ ]:
vector_new.select("features","diagnostico").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|features                                                                                                                                                                                                        |diagnostico|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|[11.08,18.83,73.3,361.6,0.1216,0.2154,0.1689,0.06367,0.2196,0.0795,0.2114,1.027,1.719,13.99,0.007405,0.04549,0.04588,0.01339,0.01738,0.004435,13.24,32.82,91.76,508.1,0.2184,0.9379,0.8402,0.2524,0.4154,0.1403]|0.0        |
+-----------------------------------------------------------------------------------------------------------

In [ ]:
resultado_new = log_reg.evaluate(vector_new).predictions
resultado_new.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+--------------------+--------------------+--------------------+----------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|d

In [ ]:
resultado_new.select(['prediction']).show()

+----------+
|prediction|
+----------+
|       1.0|
+----------+



In [ ]:
#Decision Tree

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df_classifier = DecisionTreeClassifier(labelCol="diagnostico").fit(train)
df_classifier



DecisionTreeClassificationModel: uid=DecisionTreeClassifier_22e70a6c5847, depth=5, numNodes=25, numClasses=2, numFeatures=30

In [ ]:
df_prediccion = df_classifier.transform(test)
df_prediccion.show()

+--------------------+-----------+-------------+--------------------+----------+
|            features|diagnostico|rawPrediction|         probability|prediction|
+--------------------+-----------+-------------+--------------------+----------+
|[8.219,20.7,53.27...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[8.597,18.6,54.09...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[8.878,15.49,56.7...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.295,13.9,59.96...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.465,21.01,60.1...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.504,12.44,60.3...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.606,16.84,61.6...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.668,18.1,61.06...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.683,19.34,61.0...|        0.0|  [225.0,2.0]|[0.99118942731277...|       0.0|
|[9.72,18.22,60.73...|      

In [ ]:
#Si quisieramos predecir la observacion nueva 
df_new = df_classifier.transform(vector_new)
df_new.show()

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+-----------+--------------------+-------------+-----------+----------+
|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|diagnostico|     

In [ ]:
df_new.select(['prediction']).show()

+----------+
|prediction|
+----------+
|       0.0|
+----------+



In [ ]:
#accuracy
df_accuracy = MulticlassClassificationEvaluator(labelCol="diagnostico",
                                            metricName="accuracy").evaluate(df_prediccion)

In [ ]:
df_accuracy

0.9433962264150944

In [ ]:
df_precision = MulticlassClassificationEvaluator(labelCol="diagnostico",
                                                 metricName="weightedPrecision").evaluate(df_prediccion)

In [ ]:
df_precision

0.9466414834719057

In [ ]:
df_prediccion.columns

['features', 'diagnostico', 'rawPrediction', 'probability', 'prediction']

In [ ]:
df_classifier.featureImportances

SparseVector(30, {1: 0.0784, 3: 0.0102, 5: 0.0072, 7: 0.0648, 13: 0.0042, 20: 0.1166, 27: 0.7188})

In [ ]:
df_auc = MulticlassClassificationEvaluator(labelCol="diagnostico",
                                           ).evaluate(df_prediccion)
df_auc

0.944017399471423

In [ ]:
# randomforest

from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf_classifier = RandomForestClassifier(labelCol="diagnostico",
                                       numTrees=100).fit(train)
                                       

In [ ]:
rf_prediccion = rf_classifier.transform(test)
rf_prediccion.show()

+--------------------+-----------+--------------------+--------------------+----------+
|            features|diagnostico|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|[8.219,20.7,53.27...|        0.0|[86.2251435532563...|[0.86225143553256...|       0.0|
|[8.597,18.6,54.09...|        0.0|[99.2399892502618...|[0.99239989250261...|       0.0|
|[8.878,15.49,56.7...|        0.0|[96.4066603453954...|[0.96406660345395...|       0.0|
|[9.295,13.9,59.96...|        0.0|[96.3071806421501...|[0.96307180642150...|       0.0|
|[9.465,21.01,60.1...|        0.0|[99.1565227219077...|[0.99156522721907...|       0.0|
|[9.504,12.44,60.3...|        0.0|[99.2597887490086...|[0.99259788749008...|       0.0|
|[9.606,16.84,61.6...|        0.0|[96.1183816332172...|[0.96118381633217...|       0.0|
|[9.668,18.1,61.06...|        0.0|[99.3312173204372...|[0.99331217320437...|       0.0|
|[9.683,19.34,61.0...|        0.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
rf_auc = BinaryClassificationEvaluator(labelCol="diagnostico").evaluate(rf_prediccion)
rf_auc

0.9973040977713875

In [ ]:
rf_classifier.featureImportances

SparseVector(30, {0: 0.061, 1: 0.0167, 2: 0.019, 3: 0.0787, 4: 0.0025, 5: 0.0073, 6: 0.0344, 7: 0.0899, 8: 0.0021, 9: 0.003, 10: 0.0146, 11: 0.0048, 12: 0.0122, 13: 0.0313, 14: 0.0043, 15: 0.0024, 16: 0.0074, 17: 0.0061, 18: 0.0038, 19: 0.0036, 20: 0.1418, 21: 0.0223, 22: 0.1484, 23: 0.0666, 24: 0.0109, 25: 0.0134, 26: 0.0268, 27: 0.1565, 28: 0.0055, 29: 0.0025})

In [ ]:
#Gradient Boosting 

from pyspark.ml.classification import GBTClassifier
pyspark_gradient_boosting_method = GBTClassifier(labelCol = "diagnostico", 
featuresCol = "features")

pyspark_gradient_boosting_method_fitted = pyspark_gradient_boosting_method.fit(train)

In [ ]:
pyspark_yhat = pyspark_gradient_boosting_method_fitted.transform(test)

pyspark_yhat.show()

+--------------------+-----------+--------------------+--------------------+----------+
|            features|diagnostico|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|[8.219,20.7,53.27...|        0.0|[1.53380508984279...|[0.95553674832119...|       0.0|
|[8.597,18.6,54.09...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[8.878,15.49,56.7...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[9.295,13.9,59.96...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[9.465,21.01,60.1...|        0.0|[1.53380508984279...|[0.95553674832119...|       0.0|
|[9.504,12.44,60.3...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[9.606,16.84,61.6...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[9.668,18.1,61.06...|        0.0|[1.54649203482169...|[0.95660241534170...|       0.0|
|[9.683,19.34,61.0...|        0.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
boosting_auc = BinaryClassificationEvaluator(labelCol="diagnostico").evaluate(pyspark_yhat)
boosting_auc

0.99137311286844

In [ ]:
pyspark_yhat.crosstab('diagnostico', 'prediction').show()

+----------------------+---+---+
|diagnostico_prediction|0.0|1.0|
+----------------------+---+---+
|                   1.0|  2| 50|
|                   0.0|100|  7|
+----------------------+---+---+



In [ ]:
#SVM

from pyspark.ml.classification import LinearSVC
svmm= LinearSVC(featuresCol='features', labelCol='diagnostico')
model_svm = svmm.fit(train)
print(model_svm.intercept)
print(model_svm.coefficients)

-32.62193777674548
[0.024288448391032034,0.16806857274713688,5.343469634950744e-06,-0.000443892190406948,5.439208629751781,-19.574873459766415,7.67640664855719,58.316876454960386,-12.549848742711259,-1.2435118462325065,7.466819701706055,-1.1437783318055055,0.3069376632007045,0.019070639589776006,46.61299143891963,-83.89541794770892,4.2810183910268345,156.67183797584136,-52.301784122184905,-418.3704428456192,0.404220347633343,0.23960024278645084,0.015325638632933661,0.0016881640015392499,31.335010953594196,-4.765800426581944,5.366818485572666,3.7609993315875103,10.244893868478275,64.95064856231183]


In [ ]:
svm_testing = model_svm.transform(test)
svm_testing.show()


+--------------------+-----------+--------------------+----------+
|            features|diagnostico|       rawPrediction|prediction|
+--------------------+-----------+--------------------+----------+
|[8.219,20.7,53.27...|        0.0|[8.67909504644862...|       0.0|
|[8.597,18.6,54.09...|        0.0|[16.0589976556386...|       0.0|
|[8.878,15.49,56.7...|        0.0|[9.46371647709970...|       0.0|
|[9.295,13.9,59.96...|        0.0|[7.76809065567055...|       0.0|
|[9.465,21.01,60.1...|        0.0|[6.21182150697223...|       0.0|
|[9.504,12.44,60.3...|        0.0|[9.67965267343850...|       0.0|
|[9.606,16.84,61.6...|        0.0|[8.53969255326314...|       0.0|
|[9.668,18.1,61.06...|        0.0|[7.63555676384526...|       0.0|
|[9.683,19.34,61.0...|        0.0|[7.42712974188250...|       0.0|
|[9.72,18.22,60.73...|        0.0|[17.9088927725464...|       0.0|
|[9.755,28.2,61.68...|        0.0|[5.77015666683354...|       0.0|
|[9.777,16.99,62.5...|        0.0|[8.45836603975396...|       

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
svm_testing.crosstab('diagnostico','prediction').show()

+----------------------+---+---+
|diagnostico_prediction|0.0|1.0|
+----------------------+---+---+
|                   1.0|  2| 50|
|                   0.0|105|  2|
+----------------------+---+---+



In [ ]:
#NAIVE BAYES
from pyspark.ml.classification import NaiveBayes
NB = NaiveBayes(featuresCol='features', labelCol='diagnostico')
NB_model = NB.fit(train)
NB_testing = NB_model.transform(test)
NB_testing.show()

+--------------------+-----------+--------------------+--------------------+----------+
|            features|diagnostico|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|[8.219,20.7,53.27...|        0.0|[-1095.6537204971...|[1.0,2.2885159767...|       0.0|
|[8.597,18.6,54.09...|        0.0|[-1098.1580260306...|[1.0,9.2430636824...|       0.0|
|[8.878,15.49,56.7...|        0.0|[-1229.9630984697...|[0.99999999999662...|       0.0|
|[9.295,13.9,59.96...|        0.0|[-1228.5174935252...|[0.99999999999991...|       0.0|
|[9.465,21.01,60.1...|        0.0|[-1301.9780192607...|[1.0,2.9168671787...|       0.0|
|[9.504,12.44,60.3...|        0.0|[-1188.1089860145...|[0.99999999999996...|       0.0|
|[9.606,16.84,61.6...|        0.0|[-1286.3202802062...|[1.0,9.7423094596...|       0.0|
|[9.668,18.1,61.06...|        0.0|[-1363.7975715420...|[0.99999999999997...|       0.0|
|[9.683,19.34,61.0...|        0.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
matrix_confusion_nb = NB_testing.crosstab('diagnostico','prediction')
matrix_confusion_nb.show()

+----------------------+---+---+
|diagnostico_prediction|0.0|1.0|
+----------------------+---+---+
|                   1.0| 12| 40|
|                   0.0|104|  3|
+----------------------+---+---+

